# Logistic Regression

Objectif: Titanic survived Prediction

In [2]:
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.log4j._
import org.apache.spark.ml.feature.{VectorAssembler,StringIndexer,VectorIndexer,OneHotEncoder}
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.mllib.evaluation.MulticlassMetrics

import org.apache.spark.ml.classification.LogisticRegression
import org.apache.log4j._
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer, VectorIndexer, OneHotEncoder}
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.mllib.evaluation.MulticlassMetrics


In [8]:
Logger.getLogger("org").setLevel(Level.ERROR)
val file ="Machine_Learning_Sections/Classification/titanic.csv"
val df = spark.read.option("header","true").option("inferSchema","true").format("csv").load(file)
df.printSchema

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



file: String = Machine_Learning_Sections/Classification/titanic.csv
df: org.apache.spark.sql.DataFrame = [PassengerId: int, Survived: int ... 10 more fields]


In [10]:
val logregdataall = df.select(df("Survived").as("label"), $"Pclass", $"Sex", $"Age", $"SibSp", $"Parch", $"Fare", $"Embarked")
val logregdata = logregdataall.na.drop()
logregdata.printSchema

root
 |-- label: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)



logregdataall: org.apache.spark.sql.DataFrame = [label: int, Pclass: int ... 6 more fields]
logregdata: org.apache.spark.sql.DataFrame = [label: int, Pclass: int ... 6 more fields]


In [11]:
val genderIndexer = new StringIndexer().setInputCol("Sex").setOutputCol("SexIndex")
val embarkIndexer = new StringIndexer().setInputCol("Embarked").setOutputCol("EmbarkIndex")

val genderEncoder = new OneHotEncoder().setInputCol("SexIndex").setOutputCol("SexVec")
val embarkEncoder = new OneHotEncoder().setInputCol("EmbarkIndex").setOutputCol("EmbarkVec")

genderIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_b18adc6cab7d
embarkIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_79fa96eb557c
genderEncoder: org.apache.spark.ml.feature.OneHotEncoder = oneHot_c2917011ec1d
embarkEncoder: org.apache.spark.ml.feature.OneHotEncoder = oneHot_2792bf45c188


In [26]:
val assembler = (new VectorAssembler()
                  .setInputCols(Array("Pclass", "SexVec", "Age","SibSp","Parch","Fare","EmbarkVec"))
                  .setOutputCol("features") )

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_2f580c9a532b


In [27]:
val Array(training, test) = logregdata.randomSplit(Array(0.75, 0.25), seed = 12345)
training.count

training: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: int, Pclass: int ... 6 more fields]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: int, Pclass: int ... 6 more fields]
res14: Long = 524


In [28]:
training.filter($"label"=== 0).count

res15: Long = 312


In [29]:
training.filter($"label"=== 1).count

res16: Long = 212


In [30]:
import org.apache.spark.ml.Pipeline
val lr = new LogisticRegression()
val pipeline = 
new Pipeline().setStages(Array(genderIndexer,embarkIndexer,genderEncoder,embarkEncoder,assembler, lr))

import org.apache.spark.ml.Pipeline
lr: org.apache.spark.ml.classification.LogisticRegression = logreg_2ff6bf6ace4b
pipeline: org.apache.spark.ml.Pipeline = pipeline_2f92f80e12f0


In [31]:
val model = pipeline.fit(training)

model: org.apache.spark.ml.PipelineModel = pipeline_2f92f80e12f0


In [32]:
val results = model.transform(test)

results: org.apache.spark.sql.DataFrame = [label: int, Pclass: int ... 14 more fields]


In [35]:
results.count

res19: Long = 188


In [36]:
results.select($"prediction",$"label").show(188)

+----------+-----+
|prediction|label|
+----------+-----+
|       1.0|    0|
|       1.0|    0|
|       1.0|    0|
|       1.0|    0|
|       1.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       1.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       1.0|    0|
|       0.0|    0|
|       0.0|    0|
|       1.0|    0|
|       1.0|    0|
|       1.0|    0|
|       1.0|    0|
|       1.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|

In [42]:
val prediction = results.select($"prediction",$"label").as[(Double, Double)].rdd
val metrics = new MulticlassMetrics(prediction)

prediction: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[171] at rdd at <console>:62
metrics: org.apache.spark.mllib.evaluation.MulticlassMetrics = org.apache.spark.mllib.evaluation.MulticlassMetrics@6f4b369


In [43]:
// prediction accuracy 
metrics.accuracy

res24: Double = 0.8191489361702128


In [45]:
// confusion Matrix
metrics.confusionMatrix

res26: org.apache.spark.mllib.linalg.Matrix =
100.0  12.0
22.0   54.0
